# Diet Problem

The goal of the Diet Problem is to select foods that satisfy daily nutritional requirements at minimum cost. 

This problem can be formulated as a linear program, for which constraints limit the number of calories and the amount of vitamins, minerals, fats, sodium, and cholesterol in the diet.

[Danzig (1990)](G.B. Dantzig. The Diet Problem, Interfaces 20(4), 1990, 43-47)  notes that the diet problem was motivated by the US Army's desire to minimize the cost of feeding GIs in the field while still providing a healthy diet.


<img src = 'diet2.jpg'>

<img src='diet1.png'>

## Lower and upper bound on each nutrient on daily basis

| Nutrion    |  Nmin    | Nmax     |
|--------|------|------|
| Cal     |   2000     |  |
| Carbo  |   350    |375     |
| Protein |   55     |  |
| VitA    |  100     |  |
| VitC    |  100     |  |
| Calc    |  100    |   |
| Iron    |  100   |    |

## Cost and volumn of each food per serving

| Food             | Cost | Volumn |
|------------------|------|------|
| Cheeseburger     | 1.84 | 4.0  |
| Ham Sandwich     | 2.19 | 7.5  |
| Hamburger        | 1.84 | 3.5  |
| Fish Sandwich    | 1.44 | 5.0  |
| Chicken Sandwich | 2.29 | 7.3  |
| Fries            | .77  | 2.6  |
| Sausage Biscuit  | 1.29 | 4.1  |
| Lowfat Milk      | .60  | 8.0  |
| Orange Juice     | .72  | 12.0 |


## Amount of nutrient in each food 

| Food             | Carbo | Protein | VitA | VitC | Calc | Iron | Cal |
|------------------|-------|---------|------|------|------|------|-----|
| Cheeseburger     | 510   | 34      | 28   | 15   | 6    | 30   | 20  |
| Ham Sandwich     | 370   | 35      | 24   | 15   | 10   | 20   | 20  |
| Hamburger        | 500   | 42      | 25   | 6    | 2    | 25   | 20  |
| Fish Sandwich    | 370   | 38      | 14   | 2    | 0    | 15   | 10  |
| Chicken Sandwich | 400   | 42      | 31   | 8    | 15   | 15   | 8   |
| Fries            | 220   | 26      | 3    | 0    | 15   | 0    | 2   |
| Sausage Biscuit  | 345   | 27      | 15   | 4    | 0    | 20   | 15  |
| Lowfat Milk      | 110   | 12      | 9    | 10   | 4    | 30   | 0   |
| Orange Juice     | 80    | 20      | 1    | 2    | 120  | 2    | 2   |

## Set 
F = set of food

N = set of nutrients

## Parameters
$c_i$ = cost of per serving of food $i$, $\forall i \in F$

$a_{ij}$ = amount of nutrient $j$ in food $i$, $\forall i \in F, \forall j \in N$

$Nmin_j$ = minimum level of nutrient $j$, $\forall j \in N$

$Nmax_j$ = maximum level of nutrient $j$, $\forall j \in N$

$V_i$ = the volume per serving of food $i$, $\forall i \in F$

$Vmax$ = maximum volume of food consumed


## Variables
$x_i=$ number of servings of food i to consume 

## Ojective
Minimize the total cost of the food

$\min \sum_{i \in F} c_i x_i$

## constraints
1. limit the nutritent for each nutrient:

$Nmin_j \leq  \sum_{i\in F} x_i * a_{i,j} \leq Nmax_j,   \forall j \in N $

2. Limit the volumn of the total food consumed

$\sum_{i\in F} V_i x_i \leq Vmax $

## Model formation


In [28]:
from pyomo.environ import *

In [29]:
# input data
import pandas as pd 
model = ConcreteModel()
nutrient_df = pd.read_csv("nutrient.csv", index_col = 'Food')


In [30]:
# sets
model.set_food = Set(initialize = nutrient_df.index)
model.set_nutrient = Set(initialize = nutrient_df.columns)

In [31]:
# parameters nutrient:
nutrient = nutrient_df.stack().to_dict()
model.nutrient = Param(model.set_food, model.set_nutrient,
                      initialize = nutrient)

In [32]:
# parameters： lower and upper bound nutrients
Nmin = [2000, 350, 55, 100, 100, 100, 100]    
Nmax = [None, 350, None, None, None, None, None]    

# replace None with Inf 
import numpy as np

Nmin = np.array(Nmin)
Nmax = np.array(Nmax)

print(type(Nmin))
Nmax[Nmax==None] = float('inf')

Nmin = dict(zip(nutrient_df.columns, Nmin))
Nmax = dict(zip(nutrient_df.columns, Nmax))


model.Nmin = Param(model.set_nutrient, initialize = Nmin)
model.Nmax = Param(model.set_nutrient, initialize = Nmax)


<class 'numpy.ndarray'>


In [33]:
# parameter: Volumn

max_volumn = 75

In [34]:
#parameters： Cost and Volumn of food per serving
Cost = [1.84, 2.19, 1.84, 1.44, 2.29, 0.77, 1.29, 0.60, 0.72]
Volumn = [4, 7.5, 3.5, 5, 7.3, 2.6, 4.1, 8, 12]

Cost = dict(zip(nutrient_df.index,   Cost))
Volumn = dict(zip(nutrient_df.index, Volumn))

model.Cost = Param(model.set_food, initialize = Cost)
model.Volumn = Param(model.set_food, initialize = Volumn)

In [35]:
# Variables
model.x = Var(model.set_food, domain = NonNegativeIntegers)

In [19]:
# Objetives
def _obj(model):
    return summation(model.Cost, model.x) # sum_product no *

# alternative way
# def _obj(model):
#     return sum(model.Cost[i] * model.x[i] for i in model.set_food)

model.obj = Objective(rule = _obj, sense = minimize)


In [36]:
# Constraints:
# up/low bound
def _con_nutrient(model, j):
    value = sum(model.x[i] * model.nutrient[i,j] for i in model.set_food)
#     return  model.Nmin[j] <= value <= model.Nmax[j] 
    return inequality(model.Nmin[j], value, model.Nmax[j])

model._rule_nutrient = Constraint(model.set_nutrient, rule = _con_nutrient)



In [37]:
model.pprint()

3 Set Declarations
    nutrient_index : Dim=0, Dimen=2, Size=63, Domain=None, Ordered=False, Bounds=None
        Virtual
    set_food : Dim=0, Dimen=1, Size=9, Domain=None, Ordered=False, Bounds=None
        ['Cheeseburger', 'Chicken Sandwich', 'Fish Sandwich', 'Fries', 'Ham Sandwich', 'Hamburger', 'Lowfat Milk', 'Orange Juice', 'Sausage Biscuit']
    set_nutrient : Dim=0, Dimen=1, Size=7, Domain=None, Ordered=False, Bounds=None
        ['Cal', 'Calc', 'Carbo', 'Iron', 'Protein', 'VitA', 'VitC']

5 Param Declarations
    Cost : Size=9, Index=set_food, Domain=Any, Default=None, Mutable=False
        Key              : Value
            Cheeseburger :  1.84
        Chicken Sandwich :  2.29
           Fish Sandwich :  1.44
                   Fries :  0.77
            Ham Sandwich :  2.19
               Hamburger :  1.84
             Lowfat Milk :   0.6
            Orange Juice :  0.72
         Sausage Biscuit :  1.29
    Nmax : Size=7, Index=set_nutrient, Domain=Any, Default=None, Mutable

In [21]:
# Constraints:
# volumn capacity
def _con_volumn(model):
#     return summation(model.x, model.Volumn) <= model.max_volumn
    return sum(model.x[i]*model.Volumn[i] for i in model.set_food) <= max_volumn

model._rule_volumn= Constraint(rule = _con_volumn)

In [22]:
solver=SolverFactory("cbc")
solver.solve(model,tee=True)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Sep 10 2019 

command line - /home/kon6750/miniconda3/bin/cbc -printingOptions all -import /tmp/tmpt3dyva40.pyomo.lp -stat=1 -solve -solu /tmp/tmpt3dyva40.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 9 (-1) rows, 9 (-1) columns and 76 (-1) elements
Statistics for presolved model
Original problem has 9 integers (0 of which binary)
Presolved problem has 9 integers (0 of which binary)
==== 0 zero objective 8 different
1 variables have objective of 0.6
1 variables have objective of 0.72
1 variables have objective of 0.77
1 variables have objective of 1.29
1 variables have objective of 1.44
2 variables have objective of 1.84
1 variables have objective of 2.19
1 variables have objective of 2.29
==== absolute objective values 8 different
1 variables have objective of 0.6
1 variables have objective of 0.72
1 variables have objective of 0.77
1 variables have objective of 1.29
1 variabl

{'Problem': [{'Name': 'unknown', 'Lower bound': 15.28, 'Upper bound': 15.28, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 9, 'Number of binary variables': 0, 'Number of integer variables': 9, 'Number of nonzeros': 9, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.36, 'Wallclock time': 0.22, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 11, 'Number of created subproblems': 11}, 'Black box': {'Number of iterations': 620}}, 'Error rc': 0, 'Time': 0.3208956718444824}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [23]:
model.pprint()

3 Set Declarations
    nutrient_index : Dim=0, Dimen=2, Size=63, Domain=None, Ordered=False, Bounds=None
        Virtual
    set_food : Dim=0, Dimen=1, Size=9, Domain=None, Ordered=False, Bounds=None
        ['Cheeseburger', 'Chicken Sandwich', 'Fish Sandwich', 'Fries', 'Ham Sandwich', 'Hamburger', 'Lowfat Milk', 'Orange Juice', 'Sausage Biscuit']
    set_nutrient : Dim=0, Dimen=1, Size=7, Domain=None, Ordered=False, Bounds=None
        ['Cal', 'Calc', 'Carbo', 'Iron', 'Protein', 'VitA', 'VitC']

5 Param Declarations
    Cost : Size=9, Index=set_food, Domain=Any, Default=None, Mutable=False
        Key              : Value
            Cheeseburger :  1.84
        Chicken Sandwich :  2.29
           Fish Sandwich :  1.44
                   Fries :  0.77
            Ham Sandwich :  2.19
               Hamburger :  1.84
             Lowfat Milk :   0.6
            Orange Juice :  0.72
         Sausage Biscuit :  1.29
    Nmax : Size=7, Index=set_nutrient, Domain=Any, Default=None, Mutable

In [24]:
for i in nutrient_df.index:
    print('product',model.x[i]())

product 3.0
product 0.0
product 2.0
product 0.0
product 0.0
product 4.0
product 0.0
product 5.0
product 0.0


In [25]:
# model.pprint()
# !help Varible
# float('inf')
!help inequality()

/bin/sh: 1: Syntax error: "(" unexpected


In [26]:
type(float('inf'))


float

In [27]:
help(inequality)


Help on function inequality in module pyomo.core.expr.logical_expr:

inequality(lower=None, body=None, upper=None, strict=False)
    A utility function that can be used to declare inequality and
    ranged inequality expressions.  The expression::
    
        inequality(2, model.x)
    
    is equivalent to the expression::
    
        2 <= model.x
    
    The expression::
    
        inequality(2, model.x, 3)
    
    is equivalent to the expression::
    
        2 <= model.x <= 3
    
    .. note:: This ranged inequality syntax is deprecated in Pyomo.
        This function provides a mechanism for expressing
        ranged inequalities without chained inequalities.
    
    Args:
        lower: an expression defines a lower bound
        body: an expression defines the body of a ranged constraint
        upper: an expression defines an upper bound
        strict (bool): A boolean value that indicates whether the inequality
            is strict.  Default is :const:`False`.
    
